In [2]:

import glob, sys, sqlite3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from colorama import Fore, Back, Style
from matplotlib.ticker import PercentFormatter
from matplotlib.pyplot import figure

# Noms de colonnes à utiliser
names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément', 'Élément', 'Code Produit',
         'Produit', 'Code Année', 'Année', 'Unité', 'Valeur', 'Symbole', 'Description du Symbole']

fns = glob.glob('*/*.csv')

conn = sqlite3.connect('bdd.sqlite')

dfs = {}

for fn in fns:

    fns = fn.split('/')[-1][13:-4]

    if fns == 'Groupe_de_pays':
        df = pd.read_csv(fn)
    else:
        df = pd.read_csv(fn, names=names, header=0)

    print(f" - {fn:50s} ({df.shape[0]:6d}, {df.shape[1]:2d}) - {Fore.BLACK}{Back.GREEN}{fns}{Style.RESET_ALL}")
    df.to_sql(fns, con=conn, if_exists='replace')
    dfs[fns] = df

# Récupération des données dans un sql

plt.rcParams["figure.figsize"] = (22,10)


 - data/FAOSTAT_2013_vegetal.csv                      (104871, 14) - vegetal
 - data/FAOSTAT_2013_animal.csv                       ( 37166, 14) - animal
 - data/FAOSTAT_2013_superficie.csv                   (   238, 14) - superficie
 - data/FAOSTAT_2023_Groupe_de_pays.csv               (  1941,  7) - Groupe_de_pays
 - data/FAOSTAT_2013_population.csv                   (   175, 14) - population
 - data/FAOSTAT_2013_sous_alimentation.csv            (   204, 14) - sous_alimentation
 - data/FAOSTAT_2013_cereal.csv                       ( 16057, 14) - cereal


In [52]:

animaux = pd.DataFrame(dfs['animal'])
vegetaux = pd.DataFrame(dfs['vegetal'])

# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Origine',
    'Élément',
    'Valeur'
]]

kcal = "Disponibilité alimentaire (Kcal/personne/jour)"
prot = "Disponibilité de protéines en quantité (g/personne/jour)"
poid = "Disponibilité alimentaire en quantité (kg/personne/an)"
aliments.query("Élément == @kcal or Élément == @prot or Élément == @poid", inplace=True)

print(kcal)
# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, dfs['population'][['Pays', 'Valeur']])

# Renommer les colonnes
aliments.rename(columns={
    'Disponibilité alimentaire (Kcal/personne/jour)'           : 'kcal/pers/an',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'kg prot/pers/an',
    'Disponibilité alimentaire en quantité (kg/personne/an)'   : 'kg/pers/an',
    'Valeur' : 'population'
}, inplace=True)

aliments['population'] *= 1000
aliments['kcal/pers/an'] *= 365
aliments['kg prot/pers/an'] *= 365
aliments['kg prot/pers/an'] /= 1000

aliments['énergie/poids'] = aliments['kcal/pers/an'] / aliments['kg/pers/an']
aliments['kg prot/poids'] = aliments['kg prot/pers/an'] / aliments['kg/pers/an']

# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)
aliments.replace(np.inf, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.sample(10)


Disponibilité alimentaire (Kcal/personne/jour)
Résultat :  10 colonnes,  4405 lignes


,Code Pays,Pays,Produit,Origine,kcal/pers/an,kg/pers/an,kg prot/pers/an,population,énergie/poids,kg prot/poids
3848,217,Togo,Sucre Eq Brut,Végétale,41975.0,12.10,0.00000,6817000,3469.008264,0.000000
1642,89,Guatemala,"Épices, Autres",Végétale,730.0,0.20,0.02555,15468000,3650.000000,0.127750
1816,103,Iraq,Boissons Alcooliques,Végétale,1095.0,0.40,0.00000,33765000,2737.500000,0.000000
2976,166,Panama,Crustacés,Animale,365.0,1.01,0.09125,3864000,361.386139,0.090347
1430,81,Ghana,Huile de Coco,Végétale,2920.0,0.31,0.00000,25905000,9419.354839,0.000000
2651,155,Vanuatu,Huile d'Arachide,Végétale,0.0,0.02,0.00000,253000,0.000000,0.000000
3421,181,Zimbabwe,Pois,Végétale,3285.0,0.99,0.22265,14150000,3318.181818,0.224899
1423,81,Ghana,Graines Colza/Moutarde,Végétale,0.0,0.00,0.00000,25905000,0.000000,0.000000
1290,72,Djibouti,"Poissons Marins, Autres",Animale,365.0,0.39,0.04015,873000,935.897436,0.102949
3675,207,Suriname,Riz (Eq Blanchi),Végétale,232505.0,68.68,4.60265,539000,3385.337798,0.067016


In [44]:

print("Moyenne du rapport énergie poids des oeufs (kcal/kg) :")
print(np.mean(aliments.query("Produit == 'Oeufs'")['énergie/poids']))


Moyenne du rapport énergie poids des oeufs (kcal/kg) :
1362.3173838894493


In [65]:
prod_en_poids = aliments[['Produit', 'énergie/poids']].groupby('Produit').mean().sort_values('énergie/poids', ascending=False)

prod_en_poids.head(20)

,énergie/poids
Produit,
Huile de Palme,8712.998046
Huile de Soja,8671.981316
Graisses Animales Crue,7714.200488
Huil Plantes Oleif Autr,7497.492849
Huile de Tournesol,7264.736416
"Beurre, Ghee",6607.422927
Huile de Son de Riz,6208.686869
Huile d'Olive,6134.059095
Huile de Coco,6066.931493


In [61]:
prod_prot_poids = aliments[['Produit', 'kg prot/poids']].groupby('Produit').mean().sort_values('kg prot/poids', ascending=False)

prod_prot_poids.head(20)

,kg prot/poids
Produit,
Arachides Decortiquees,0.245061
Soja,0.230789
Haricots,0.202210
Légumineuses Autres,0.197367
Pois,0.192526
Abats Comestible,0.176632
"Viande, Autre",0.160918
Viande de Bovins,0.146130
Viande d'Ovins/Caprins,0.137785


In [68]:
df = dfs['vegetal'][['Élément', 'Produit', 'Valeur']]

df = df.query("Élément == 'Disponibilité intérieure'").groupby('Produit').sum()

df = pd.merge(df, prod_en_poids, left_index = True, right_index = True)

df.rename(columns={'Valeur' : ' poids (kg)', 'énergie/poids' : 'kcal/kg'})
df['poids (kg)'] *= 1000000
df['énergie (kcal)'] = df['poids (kg)'] * df['kcal/kg']
df.head(5)

KeyError: 'poids (kg)'